# IMPORTING THE LIBRARIES 

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import statistics
import os

###############################################
from peakutils import indexes
from peakutils import baseline
from scipy.signal import find_peaks as fp
from scipy.signal import savgol_filter 
###############################################
from bokeh.plotting import figure , show
from pybaselines import whittaker as pl



from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

# DATA ARRANGEMENT

In the main directory we can see that there are 8 subfolders. 

Each subfolder have almost 12 spectras per sample , the idea behind that would be , instead of having just one spectra per sample , and to just rely on one information , its always better have to multiple measurements per samples , and then this could be used for building the Calibration Model

Instead of having 12 different csv per samples , its always good to have a single dataframe -> This new dataframe will have 1st column as wavelength , and 2nd -13th column as Intensities 

In [3]:
def load_data(folder_path):
    # List to store DataFrames for intensity columns
    intensity_dfs = []

    # List to store CSV file names
    csv_file_names = []

    # Loop through each file in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            
            # Read CSV file into a DataFrame
            df = pd.read_csv(file_path)
            
            # Extract intensity column and store it in the list
            intensity_dfs.append(df.iloc[:, 1])  # Assuming intensity is in the second column
            
            # Store CSV file name
            csv_file_names.append(os.path.splitext(file_name)[0])

    # Read the wavelength column from the first CSV file
    wavelength_df = pd.read_csv(os.path.join(folder_path, os.listdir(folder_path)[1]), usecols=[0])

    # Concatenate wavelength column with intensity columns
    result_df = pd.concat([wavelength_df] + intensity_dfs, axis=1)

    # Rename the columns with CSV file names
    column_names = ['wavelength'] + [f'{csv_file_names[i]}' for i in range(len(intensity_dfs))]

    result_df.columns = column_names
    # result_df.reset_index(drop=True, inplace=True)

    return result_df

Loading the dataframe.

In [4]:
CR210LA_df = load_data('CR210LA')
###################################################
CR240LA_df = load_data('CR240LA')
###################################################
CR300LA_df = load_data('CR300LA')
###################################################
CR440Y_df = load_data('CR440Y')
###################################################
CR570CP_df = load_data('CR570CP')
###################################################
CR700Y_df = load_data('CR700Y')
###################################################
HR660Y_df = load_data('HR660Y')
# ###################################################
# folder_path = 'CR1000Y'
# CR1000Y_df = load_data(folder_path)

# Peak Selection and Data Trimming

The dataframe  which we have is very big ,it could be trimmed now according to the wavelength , by adjusting two parameters "Wavelength_Min" , "Wavelength_Max"

In [5]:
Wavelength_Min =201.5
Wavelength_Max = 204

CR210LA_Select_df = CR210LA_df[(CR210LA_df['wavelength'] >= Wavelength_Min) & (CR210LA_df['wavelength'] <= Wavelength_Max)]
CR240LA_Select_df = CR240LA_df[(CR240LA_df['wavelength'] >= Wavelength_Min) & (CR240LA_df['wavelength'] <= Wavelength_Max)]
CR300LA_Select_df = CR300LA_df[(CR300LA_df['wavelength'] >= Wavelength_Min) & (CR300LA_df['wavelength'] <= Wavelength_Max)]
CR440Y_Select_df = CR440Y_df[(CR440Y_df['wavelength'] >= Wavelength_Min) & (CR440Y_df['wavelength'] <= Wavelength_Max)]
CR570CP_Select_df = CR570CP_df[(CR570CP_df['wavelength'] >= Wavelength_Min) & (CR570CP_df['wavelength'] <= Wavelength_Max)]
CR700Y_Select_df = CR700Y_df[(CR700Y_df['wavelength'] >= Wavelength_Min) & (CR700Y_df['wavelength'] <= Wavelength_Max)]
HR660Y_Select_df = HR660Y_df[(HR660Y_df['wavelength'] >= Wavelength_Min) & (HR660Y_df['wavelength'] <= Wavelength_Max)]
# CR1000Y_Select_df = CR1000Y_df[(CR1000Y_df['wavelength'] >= Wavelength_Min) & (CR1000Y_df['wavelength'] <= Wavelength_Max)]

CR210LA_Select_df


,wavelength,CR210LA_10,CR210LA_11,CR210LA_8,CR210LA_9,CR210LA_12,CR210LA_4,CR210LA_5,CR210LA_7,CR210LA_6,CR210LA_2,CR210LA_3,CR210LA_1
646,201.533333,1025.952478,1033.890106,719.475387,990.262225,1065.233181,671.219401,665.703105,688.262670,657.789771,593.862076,674.488839,745.695664
647,201.566667,1096.264933,1089.925874,748.603091,1035.585640,1130.023378,704.277424,698.743851,721.663056,688.508578,618.405769,707.590869,788.541605
648,201.600000,1100.566806,1116.871774,772.639064,1073.089759,1143.518952,727.761211,722.626058,746.420827,712.469112,638.759180,731.552495,820.622997
649,201.633333,1073.443566,1126.255441,798.854892,1102.759156,1131.066840,745.904041,741.274392,766.746375,734.091592,659.842725,750.783801,843.045396
650,201.666667,1053.326920,1131.346792,820.778926,1125.079573,1121.046079,754.538583,750.671220,778.172348,748.572245,677.355355,760.429652,852.259795
...,...,...,...,...,...,...,...,...,...,...,...,...,...
716,203.866667,817.327783,932.250251,683.449367,957.124225,893.644345,611.629193,607.352290,631.225787,617.774948,576.874441,612.454927,691.288018
717,203.900000,817.888732,943.248019,652.751758,972.540891,900.292943,579.040890,576.300907,599.034983,589.238982,549.822691,578.547737,652.293496
718,203.933333,927.379629,1056.208101,640.800893,1069.820502,1022.441858,559.652965,560.886494,582.759425,575.262918,538.581006,558.078608,626.541569
719,203.966667,1187.180123,1304.243229,739.399267,1269.622708,1301.721642,652.256055,656.415461,679.440723,664.985654,615.398220,651.481247,724.124598


Lets plot a line plot ,to get a better picture 

In [6]:
Selected_df_Plot = figure(title = 'Selected Data S' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_1 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_2 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_3 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_4 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_5 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_6 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_7 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_8 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_9 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_10 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_11 , line_width = 2, color ="red" )
Selected_df_Plot.line(CR210LA_Select_df.wavelength,CR210LA_Select_df.CR210LA_12 , line_width = 2, color ="red" )

######################################################################################################################
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_1 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_2 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_3 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_4 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_5 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_6 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_7 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_8 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_9 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_10 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_11 , line_width = 2, color ="green" )
Selected_df_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_12 , line_width = 2, color ="green" )

###############################################################################################################

Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_1 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_2 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_3 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_4 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_5 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_6 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_7 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_8 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_9 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_10 , line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_11, line_width = 2, color ="pink")
Selected_df_Plot.line(CR300LA_Select_df.wavelength,CR300LA_Select_df.CR300LA_12, line_width = 2, color ="pink")

######################################################################################################################

Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_1 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_2 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_3 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_4 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_5 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_6 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_7 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_8 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_9 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_10 , line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_11, line_width = 2, color ="yellow")
Selected_df_Plot.line(CR440Y_Select_df.wavelength,CR440Y_Select_df.CR440Y_12, line_width = 2, color ="yellow")

##############################################################################################################

Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_1 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_2 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_3 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_4 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_5 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_6 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_7 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_8 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_9 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_10 , line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_11, line_width = 2, color ="blue")
Selected_df_Plot.line(CR570CP_Select_df.wavelength,CR570CP_Select_df.CR570CP_12, line_width = 2, color ="blue")

###############################################################################################################

Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_1 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_2 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_3 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_4 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_5 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_6 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_7 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_8 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_9 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_10 , line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_11, line_width = 2, color ="orange")
Selected_df_Plot.line(CR700Y_Select_df.wavelength,CR700Y_Select_df.CR700Y_12, line_width = 2, color ="orange")


####################################################################################################################

Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_1 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_2 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_3 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_4 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_5 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_6 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_7 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_8 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_9 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_10 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_11 , line_width = 2, color ="brown")
Selected_df_Plot.line(HR660Y_Select_df.wavelength,HR660Y_Select_df.HR660Y_12 , line_width = 2, color ="brown")


###################################################################################################################

# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_1 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_2 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_3 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_4 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_5 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_6 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_7 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_8 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_9 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_10 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_11 , line_width = 2, color ="black")
# Selected_df_Plot.line(CR1000Y_Select_df.wavelength,CR1000Y_Select_df.CR1000Y_12 , line_width = 2, color ="black")

###################################################################################################################

Selected_df_Plot.width = 600
Selected_df_Plot.height = 500
show(Selected_df_Plot)

# Data Preprocessing of the Spectra

The above plot eventhough a spectra , is still a Raw Spectra , which still has lot of Artifects , before proceeding for the Univariate Calibration , its important to Pre Process the Raw Spectra accordingly. Various Pre Processing Techniques could be used here :- 

1) Baseline Correction - Very Very little  background radiation is still present in the spectra, which corresponds to the spectral baseline and imposes difficulties for quantitative elemental analysis.
2) Normalization - Its usually noticed that , for a measurement of a similar sample , the spectra obtained for them are quite different , but there could me many reasons for it , like laser energy fluctuations , material surface , even though is homegnous isnt same overall the surface

In [7]:
def baseline_correction(df):
    """
    Perform baseline correction on the intensity columns of the input DataFrame and create a new DataFrame with corrected values.
    
    Parameters:
        df (DataFrame): Input DataFrame containing the wavelength and intensity columns.
        
    Returns:
        DataFrame: New DataFrame with baseline-corrected intensity columns and the same wavelength column as the input DataFrame.
    """
    # Copy the 'wavelength' column from the input DataFrame
    new_df = pd.DataFrame({'wavelength': df['wavelength']})
    
    # Perform baseline correction for each intensity column and add them to the new DataFrame
    for col in df.columns[1:]:  # Exclude the 'wavelength' column
        baseline, _ = pl.drpls(df[col])
        corrected_values = df[col] - baseline
        new_df[col] = corrected_values
    
    return new_df

In [8]:
CR210LA_BaselineCorrected = baseline_correction(CR210LA_Select_df)
CR240LA_BaselineCorrected = baseline_correction(CR240LA_Select_df)
CR300LA_BaselineCorrected = baseline_correction(CR300LA_Select_df)
CR440Y_BaselineCorrected = baseline_correction(CR440Y_Select_df)
CR570CP_BaselineCorrected = baseline_correction(CR570CP_Select_df)
CR700Y_BaselineCorrected = baseline_correction(CR700Y_Select_df)
HR660Y_BaselineCorrected = baseline_correction(HR660Y_Select_df)

# CR1000Y_BaselineCorrected = baseline_correction(CR1000Y_Select_df)

In [12]:
Baseline_Correction_Plot = figure(title = 'Baseline Correction' , x_axis_label = 'Wavelength' , y_axis_label = 'Intensity')

Baseline_Correction_Plot.line(CR240LA_Select_df.wavelength,CR240LA_Select_df.CR240LA_10 , line_width = 2, color ="red" )
Baseline_Correction_Plot.line(CR240LA_BaselineCorrected.wavelength,CR240LA_BaselineCorrected.CR240LA_10 , line_width = 2, color ="green" )

Baseline_Correction_Plot.width = 600
Baseline_Correction_Plot.height = 500
show(Baseline_Correction_Plot)